In [4]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk import wsd
from nltk.corpus import wordnet as wn
from spacy.cli import download
from spacy import load
import warnings
import numpy as np

In [5]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df = df.drop('id',axis = 1)

In [8]:
df = df.dropna()

In [9]:
df.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

Preprocessing of data

In [10]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('wordnet2022')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/wordnet2022.zip.
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [11]:
nlp = load('en_core_web_sm')

In [12]:
! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.

In [13]:
print(set(stopwords.words('english')))

{'own', 'doesn', 'more', "won't", 'that', 'was', 'didn', 'o', 'it', 'themselves', 'does', 'again', "don't", "wasn't", 'him', "wouldn't", 'can', 'few', 'ain', 'each', 'been', 'as', 'by', "you'll", 'whom', 'some', 'on', 'my', 'i', 'shan', 'above', "mightn't", 'he', 'because', "shouldn't", 'them', "it's", 'doing', 're', 'be', 'same', "shan't", 'but', 'up', 'we', 'very', "should've", 'hasn', "mustn't", 'is', 'both', 'aren', 'mightn', 'than', 'below', 'here', 'did', 'too', 'against', "couldn't", "weren't", 'your', 'any', 'these', 'the', 'won', "you'd", 'so', 'ma', 'until', 'of', 'wasn', 'after', 'why', 'further', 'other', 'this', 's', 'only', 'and', 'between', "you've", 'her', 'yourselves', 'out', 'then', 'hers', 'to', "isn't", 'down', 'into', 't', 'under', 'or', 'itself', "that'll", "needn't", 'will', 'our', 'who', 'yourself', 'himself', 'from', 'not', 'its', 'shouldn', 'wouldn', 'having', 'his', 'had', 'me', 'now', 'all', 'through', 'do', 'needn', 'once', "hasn't", 'most', 've', 'am', 'ab

In [12]:
messages=df.copy()
messages

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [13]:
messages.reset_index(inplace=True)
messages

,index,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
18280,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
18281,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
18282,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
18283,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [14]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    corpus = []
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    processed_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    processed_text = ' '.join(processed_words)
    return processed_text

df['title'] = df['title'].apply(preprocess_text)
df['text'] = df['text'].apply(preprocess_text)


KeyboardInterrupt



In [ ]:
df.head()

In [ ]:
import nltk
from nltk.corpus import stopwords

import re

def preprocess_corpus(messages):
    corpus = []
    for i in range(0, len(messages)):
        review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
        review = review.lower()
        review = review.split()

        review = [word for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
        
    return corpus   

In [61]:
preprocessed_corpus = preprocess_corpus(messages)

In [62]:
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X=tfidf_v.fit_transform(preprocessed_corpus).toarray()

In [63]:
y=messages['label']

In [64]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [68]:
input_dim = X_train.shape[1]
input_dim

5000

In [71]:
import tensorflow as tf
from tensorflow import keras

# Define the architecture of the DNN model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(5000,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)
# Fit the model to the training data
#model.fit(X_train, y_train, batch_size=32, epochs=10)



# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)


Epoch 1/10
307/307 [==============================] - 4s 10ms/step - loss: 0.2419 - accuracy: 0.8914 - val_loss: 0.1392 - val_accuracy: 0.9437
Epoch 2/10
307/307 [==============================] - 3s 9ms/step - loss: 0.0522 - accuracy: 0.9839 - val_loss: 0.1406 - val_accuracy: 0.9490
Epoch 3/10
307/307 [==============================] - 3s 8ms/step - loss: 0.0144 - accuracy: 0.9966 - val_loss: 0.1665 - val_accuracy: 0.9494
Epoch 4/10
307/307 [==============================] - 3s 8ms/step - loss: 0.0062 - accuracy: 0.9987 - val_loss: 0.1837 - val_accuracy: 0.9527
Epoch 5/10
307/307 [==============================] - 3s 9ms/step - loss: 0.0027 - accuracy: 0.9997 - val_loss: 0.2009 - val_accuracy: 0.9543
Epoch 6/10
307/307 [==============================] - 3s 9ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.2121 - val_accuracy: 0.9547
Epoch 7/10
307/307 [==============================] - 3s 9ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.2257 - val_accuracy: 0.9531
Epoch

In [78]:
from sklearn.metrics import accuracy_score

# Predict the labels for the test data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


189/189 [==============================] - 1s 3ms/step
Accuracy: 0.9514498757249379


In [74]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Predict the probabilities for the test data
y_pred_probs = model.predict(X_test)
# Convert probabilities to binary predictions
y_pred = np.round(y_pred_probs).astype(int)

# Convert the predicted labels and true labels to 1D arrays
y_pred = np.squeeze(y_pred)
y_true = np.squeeze(y_test)

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

print("Confusion Matrix:")
print(cm)


189/189 [==============================] - 1s 3ms/step
Confusion Matrix:
[[3244  145]
 [ 148 2498]]


In [77]:
model.save('./model_1.h5')